<a href="https://colab.research.google.com/github/Shurui-Zhang/Deep_learning/blob/main/Lab5ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset

class MyDataset(Dataset):
  def __init__(self, size=5000, dim=40, random_offset=0):
        super(MyDataset, self).__init__()
        self.size = size
        self.dim = dim
        self.random_offset = random_offset

  def __getitem__(self, index):
      if index >= len(self):
          raise IndexError("{} index out of range".format(self.__class__.__name__))

      rng_state = torch.get_rng_state()
      torch.manual_seed(index + self.random_offset)

      while True:
        img = torch.zeros(self.dim, self.dim)
        dx = torch.randint(-10,10,(1,),dtype=torch.float)
        dy = torch.randint(-10,10,(1,),dtype=torch.float)
        c = torch.randint(-20,20,(1,), dtype=torch.float)

        params = torch.cat((dy/dx, c))
        xy = torch.randint(0,img.shape[1], (20, 2), dtype=torch.float)
        xy[:,1] = xy[:,0] * params[0] + params[1]

        xy.round_()
        xy = xy[ xy[:,1] > 0 ]
        xy = xy[ xy[:,1] < self.dim ]
        xy = xy[ xy[:,0] < self.dim ]

        for i in range(xy.shape[0]):
          x, y = xy[i][0], self.dim - xy[i][1]
          img[int(y), int(x)]=1
        if img.sum() > 2:
          break

      torch.set_rng_state(rng_state)
      return img.unsqueeze(0), params

  def __len__(self):
      return self.size

train_data = MyDataset()
val_data = MyDataset(size=500, random_offset=33333)
test_data = MyDataset(size=500, random_offset=99999)

In [ ]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

In [ ]:
# automatically reload external modules if they change
%load_ext autoreload
%autoreload 2

import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchbearer
from torch import nn
from torch import optim
from torchbearer import Trial
from torch.utils.data import DataLoader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
trainloader = DataLoader(train_data, batch_size=128, shuffle=True)
testloader = DataLoader(test_data, batch_size=128, shuffle=True)
for data in trainloader:
  print(data[0].shape)
  print(data[1].shape)
  break

torch.Size([128, 1, 40, 40])
torch.Size([128, 2])


In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 48, (3, 3), stride=1, padding=1) #concolution2D layer 1为输入通道数，32为输出通道数，32也表示有32个卷积核
        self.fc1 = nn.Linear(76800, 128) #对应第5. flatten了tensor所以输入大小为32 * 12**2
        self.fc2 = nn.Linear(128, 2)#对应第6
            
    def forward(self, x):
        # 训练数据x为torch.Size([128, 1, 28, 28])，以下所有128都是一个batch中的data个数,1表示1通道
        out = self.conv1(x) #out torch.Size([128, 32, 24, 24]) 28*28通过5*5的卷积核卷积后得到24*24 
        out = F.relu(out)  
        out = out.view(out.shape[0], -1) #torch.Size([128, 4608])    
        out = self.fc1(out)#torch.Size([128, 128])        
        out = F.relu(out)
        out = self.fc2(out) #torch.Size([128, 10])
        return out

In [ ]:
# build the model
model = SimpleCNN()

# define the loss function and the optimiser
#loss_function = nn.MSELoss()
#loss_function = nn.L1Loss()#{'test_loss': 3.1808531284332275, 'test_acc': 0.8299999833106995}
loss_function = nn.SmoothL1Loss()#{'test_loss': 2.9307379722595215, 'test_acc': 0.8399999737739563}
optimiser = optim.Adam(model.parameters())

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader, test_generator=testloader)
trial.run(epochs=100)
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)

In [ ]:
class GlobalCNN(nn.Module):
    def __init__(self):
        super(GlobalCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 48, (3, 3), stride=1, padding=1) #concolution2D layer 1为输入通道数，32为输出通道数，32也表示有32个卷积核
        self.conv2 = nn.Conv2d(48, 48, (3, 3), stride=1, padding=1)
        self.fc1 = nn.Linear(48, 128) #对应第5. flatten了tensor所以输入大小为32 * 12**2
        self.fc2 = nn.Linear(128, 2)#对应第6
            
    def forward(self, x):
        # 训练数据x为torch.Size([128, 1, 28, 28])，以下所有128都是一个batch中的data个数,1表示1通道
        out = self.conv1(x) #out torch.Size([128, 32, 24, 24]) 28*28通过5*5的卷积核卷积后得到24*24 
        out = F.relu(out)  
        out = self.conv2(out) #out torch.Size([128, 32, 24, 24]) 28*28通过5*5的卷积核卷积后得到24*24 
        out = F.relu(out) 

        global_max = nn.AdaptiveMaxPool2d((1, 1))
        out = global_max(out)

        out = out.view(out.shape[0], -1) #torch.Size([128, 4608])    
        out = self.fc1(out)#torch.Size([128, 128])        
        out = F.relu(out)
        out = self.fc2(out) #torch.Size([128, 10])
        return out

In [ ]:
# build the model
model = GlobalCNN()

# define the loss function and the optimiser
#loss_function = nn.MSELoss()
#loss_function = nn.L1Loss()#{'test_loss': 3.1808531284332275, 'test_acc': 0.8299999833106995}
loss_function = nn.SmoothL1Loss()#{'test_loss': 2.9307379722595215, 'test_acc': 0.8399999737739563}
optimiser = optim.Adam(model.parameters())

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader, test_generator=testloader)
trial.run(epochs=100)
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)

In [ ]:
class AdvancedCNN(nn.Module):
    def __init__(self):
        super(AdvancedCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 48, (3, 3), stride=1, padding=1) #concolution2D layer 1为输入通道数，32为输出通道数，32也表示有32个卷积核
        self.conv2 = nn.Conv2d(48, 48, (3, 3), stride=1, padding=1)
        self.fc1 = nn.Linear(48, 128) #对应第5. flatten了tensor所以输入大小为32 * 12**2
        self.fc2 = nn.Linear(128, 2)#对应第6
            
    def forward(self, x):
        idxx = torch.repeat_interleave(torch.arange(-20, 20, dtype=torch.float).unsqueeze(0)/40, repeats=40, dim=0).to(x.device)  
        idxy = idxx.clone().t()
        idx = torch.stack([idxx, idxy]).unsqueeze(0)
        idx = torch.repeat_interleave(idx, repeats=x.shape[0], dim=0)
        x = torch.cat([x, idx], dim=1)
        out = self.conv1(x) #out torch.Size([128, 32, 24, 24]) 28*28通过5*5的卷积核卷积后得到24*24 
        out = F.relu(out)  
        out = self.conv2(out) #out torch.Size([128, 32, 24, 24]) 28*28通过5*5的卷积核卷积后得到24*24 
        out = F.relu(out) 

        global_max = nn.AdaptiveMaxPool2d((1, 1))
        out = global_max(out)

        out = out.view(out.shape[0], -1) #torch.Size([128, 4608])    
        out = self.fc1(out)#torch.Size([128, 128])        
        out = F.relu(out)
        out = self.fc2(out) #torch.Size([128, 10])
        return out

In [ ]:
# build the model
model = AdvancedCNN()

# define the loss function and the optimiser
#loss_function = nn.MSELoss()
#loss_function = nn.L1Loss()#{'test_loss': 3.1808531284332275, 'test_acc': 0.8299999833106995}
loss_function = nn.SmoothL1Loss()#{'test_loss': 2.9307379722595215, 'test_acc': 0.8399999737739563}
optimiser = optim.Adam(model.parameters())

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader, test_generator=testloader)
trial.run(epochs=10)
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)